In [102]:
#load packages
import pandas as pd 
import numpy as np
import fuzzymatcher

In [103]:
#load data and convert dates to datetime
rw = pd.read_csv("../Datasets/RW/RW_tail_numbers(converted to min and 24-hour format).csv", parse_dates=['dates'])
rw = rw.rename({'NAME':'medical_service'}, axis=1)

# load heliports data
heliports = pd.ExcelFile("..//Datasets/Airports/Heliports.xlsx", engine='openpyxl')
heliports = heliports.parse(0)[['Loc Id', 'Facility Type', 'State Id', 'County', 'City', 'Name', 'ARP Latitude Sec', 'ARP Longitude Sec']]

# load heliports data
local_airports = pd.ExcelFile("..//Datasets/Airports/local-airport-data.xlsx", engine='openpyxl')
local_airports = local_airports.parse(0)[['Loc Id', 'Facility Type', 'State Id', 'County', 'City', 'Name', 'ARP Latitude Sec', 'ARP Longitude Sec']]

In [104]:
location = pd.concat([heliports,local_airports])

In [105]:
# split origin states
rw['origin_state'] = rw.origin.str.split(", ", n = 1, expand = True)[1]
rw.origin = rw.origin.str.split(", ", n = 1, expand = True)[0]
rw = rw.iloc[:,:4].join([rw.iloc[:,-1:],rw.iloc[:,4:-1]])

# split destination states
rw['destination_state'] = rw.destination.str.split(", ", n = 1, expand = True)[1]
rw.destination = rw.destination.str.split(", ", n = 1, expand = True)[0]
rw = rw.iloc[:,:6].join([rw.iloc[:,-1], rw.iloc[:,6:-1]])

In [106]:
# replace abbreviations to full name for the best matching
rw = rw.replace(regex={r'Muni': 'Municipal', r'Rgnl':'Regional', r'Intl':'International', r"Int'l":'International',
                 r'Trml':'Terminal', r'Fld':'Field'})

location = location.replace(regex={r'Muni': 'Municipal', r'Rgnl':'Regional', r'Intl':'International', r"Int'l":'International',
                 r'Trml':'Terminal', r'Fld':'Field'})

In [107]:
# capitalize origin and destination words for the best matching
rw.origin = rw.origin.str.upper()
rw.destination = rw.destination.str.upper()

In [108]:
# add origin information
rw = fuzzymatcher.fuzzy_left_join(rw, location.add_suffix('_origin'), left_on = ['origin_state','origin'], 
                                  right_on = ['State Id_origin', 'City_origin']).iloc[:,3:]

In [109]:
# add destination information
rw = fuzzymatcher.fuzzy_left_join(rw, location.add_suffix('_destination'), left_on = ['destination_state','destination'], 
                                  right_on = ['State Id_destination', 'City_destination']).iloc[:,3:]

In [110]:
rw[['ARP Latitude Sec_origin', 'ARP Longitude Sec_origin', 'ARP Latitude Sec_destination', 'ARP Longitude Sec_destination']] = \
rw[['ARP Latitude Sec_origin', 'ARP Longitude Sec_origin', 'ARP Latitude Sec_destination', 'ARP Longitude Sec_destination']] \
.replace(regex={'N':'','W':''}).astype('float')/3600

rw[['ARP Longitude Sec_origin', 'ARP Longitude Sec_destination']] = rw[['ARP Longitude Sec_origin', 'ARP Longitude Sec_destination']]*(-1)

In [ ]:
# check matching by existing states
rw['origin_matching'] = np.where((rw['origin_state'].notna()) & (rw['origin_state'] != rw['State Id_origin']), "don't match", "match")
rw['destination_matching'] = np.where((rw['destination_state'].notna()) & (rw['destination_state'] != rw['State Id_destination']), "don't match", "match")

In [124]:
rw.to_csv('../Datasets/RW/RW_with_heliports.csv', index=False)

In [118]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125697 entries, 0 to 1656034
Data columns (total 30 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   tail_number                    125697 non-null  object        
 1   dates                          125697 non-null  datetime64[ns]
 2   aircraft                       118770 non-null  object        
 3   origin                         125687 non-null  object        
 4   origin_state                   58998 non-null   object        
 5   destination                    125696 non-null  object        
 6   destination_state              49237 non-null   object        
 7   departure                      125697 non-null  object        
 8   dep_time_zone                  125697 non-null  object        
 9   arrival                        125697 non-null  object        
 10  arr_time_zone                  125697 non-null  object        
 11 